<h3>Imports</h3>

In [1]:
from ocpa.objects.log.importer.csv import factory as ocel_import_factory
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.visualization.log.variants import factory as variants_visualization_factory

D:\Program Files\Anaconda\lib\site-packages\pm4py\objects\petri\__init__.py:19: UserWarning: please use the pm4py.objects.petri_net package instead.
  warnings.warn('please use the pm4py.objects.petri_net package instead.')
D:\Program Files\Anaconda\lib\site-packages\pm4py\objects\petri\obj.py:19: UserWarning: pm4py.objects.petri.obj is deprecated. Please use the class pm4py.objects.petri_net.obj instead.
  warnings.warn("pm4py.objects.petri.obj is deprecated. Please use the class pm4py.objects.petri_net.obj instead.")
D:\Program Files\Anaconda\lib\site-packages\pm4py\objects\petri\exporter\__init__.py:21: UserWarning: please use the pm4py.objects.petri_net.exporter package instead.
  warnings.warn('please use the pm4py.objects.petri_net.exporter package instead.')
D:\Program Files\Anaconda\lib\site-packages\pm4py\objects\petri\importer\__init__.py:21: UserWarning: please use the pm4py.objects.petri_net.importer package instead.
  warnings.warn('please use the pm4py.objects.petri_net.i

<h3>Import Test OCEL File</h3>
OCEL file downloadable from `http://ocel-standard.org/`. However, for testing a small OCEL file containing two process executions is used.

In [3]:
filename = "OCEL_example.jsonocel"
ocel = ocel_import_factory.apply(filename)

<h3>Variant Statistics</h3>

In [4]:
print("Number of process executions: "+str(len(ocel.process_executions)))
print("Number of variants: "+str(len(ocel.variants)))
print("The variants: " + str(ocel.variants))

Number of process executions: 2
Number of variants: 2
The variants: ['d8bd21497c25849b975ad8ad250a1787', '87dbcc253e53fc464e31e0ad32a182d8']


<h3>Variant Layouting</h3>

In [5]:
variant_layouting = variants_visualization_factory.apply(ocel)
print(variant_layouting[ocel.variants[0]])

([['A', [[0, 0], [2]]], ['B', [[1, 1], [2]]], ['E', [[0, 1], [1, 0]]], ['C', [[2, 2], [0, 1, 2]]], ['D', [[3, 3], [2]]], ['F', [[3, 3], [1, 0]]], ['G', [[4, 4], [0]]]], {0: ('items', 'items_1'), 1: ('items', 'items_2'), 2: ('orders', 'orders_1')})


In [6]:
print(variant_layouting[ocel.variants[1]])

([['A', [[0, 0], [1]]], ['D', [[3, 3], [1]]], ['B', [[1, 1], [1]]], ['C', [[2, 2], [1, 0]]]], {0: ('items', 'items_1'), 1: ('orders', 'orders_1')})


In [7]:
class DualSharedSubProcess:
    
class AtomicSharedActivity(DualSharedSubProcess):
    

IndentationError: expected an indented block (<ipython-input-7-e30016ef2168>, line 3)

<h2>Order-Preserving Shared Activities</h2>

In [46]:
def MaxSharedActivities(subvariant1, subvariant2):
    # Provisional data structures:
    # Input - subvariant1, subvariant2: lists
    # Output - result: Dict<ActivityName:Index1, Index2>
    
    # Initialization, copy elements
    activitySets = []
    l1 = subvariant1
    
    # Retrieves all common sublists that ensure the precise order
    for i in range(len(l1)):
        
        # Keeping track of the original indices and the remaining sublists
        l2 = subvariant2
        l2I = list(range(0,len(subvariant2)))
        set = dict()
        
        # For each activity in subvariant1 as starting point find the list of activities
        # that appear in both variants in the same order
        for j in range(i,len(l1)):
            if (l1[j] in l2):
                set[l1[j]]=(j,l2I[l2.index(l1[j])])
                l2I = l2I[l2.index(l1[j])+1:]
                l2 = l2[l2.index(l1[j])+1:]
        activitySets.append(set)
        
    # Return the maximal list of activities for optimality
    return max(activitySets, key=len)     

In [55]:
print(MaxSharedActivities(["A","D","A","D","B"],["E","D","B"]))


{'D': (1, 1), 'B': (4, 2)}
['D', 'B']


<h2>Optional Activities in Super-Variant</h2>

In [60]:
def OptionalActivity(subvariant1, subvariant2):
    
    # Provisional data structures:
    # Input - subvariant1, subvariant2: lists
    # Output - result: Dict<Index:ActivityName, Otp Indicator>
    result = dict()
    
    # Determine common activities of both variants
    intersection = MaxSharedActivities(subvariant1, subvariant2)
    
    # If the variants share no common variants, then they must represent choices, return
    if (not len(intersection)):
            return dict
            
    else :
        print("The variants both contain the following common activities: " + str(intersection))
        
    #Initiate indices
    posInSuper = 0
    startIndex1 = 0
    startIndex2 = 0
    currentIndex1 = 0
    currentIndex2 = 0
        
    # For each commonly shared activities, check the subprocesses between the current
    # activity and previous shared activity.
    for i in range(len(intersection)):
        
        #TODO always takes the first element
        # Determine the corresponding indices of the current shared activity in both variants
        currentIndex1 = intersection[list(intersection.keys())[i]][0]
        currentIndex2 = intersection[list(intersection.keys())[i]][1]
        
        # Determine the corresponding preceeding subprocesses between to shared activities
        preceeding1 = subvariant1[startIndex1:currentIndex1]
        preceeding2 = subvariant2[startIndex2:currentIndex2]    
        print("Non-matching preceeding subprocess of variant 1: " + str(preceeding1))
        print("Non-matching preceeding subprocess of variant 2: " + str(preceeding2))
        
        # If one of the subprocesses is empty, then add the other subprocess as optional
        if(preceeding1==[]):
            # Add every activity as atomic to the supervariant and as optional
            for i in range(startIndex2,currentIndex2):
                result[posInSuper] = (subvariant2[i],"Opt")
                posInSuper+=1 
            # Add the common activity to the supervariant, not as optional    
            result[posInSuper] = (subvariant2[currentIndex2],"NOpt")
            posInSuper+=1
        
        # If the other subprocesses is empty, then add the first subprocess as optional
        elif(preceeding2==[]):
            # Add every activity as atomic to the supervariant and as optional
            for i in range(startIndex1,currentIndex1):
                result[posInSuper] = (subvariant1[i],"Opt")
                posInSuper+=1  
            # Add the common activity to the supervariant, not as optional 
            result[posInSuper] = (subvariant1[currentIndex1],"NOpt")
            posInSuper+=1
            
        else:
            # In this case, neither one or the other subprocesses is optional
            # we must summarize the underlying process
            return result

        startIndex1 = currentIndex1+1
        startIndex2 = currentIndex2+1
        
    # Variant 1 has remaining activities while variant 2 does not
    if(currentIndex1<len(subvariant1)-1 and currentIndex2==len(subvariant2)-1):
        for i in range(startIndex1,len(subvariant1)):
                result[posInSuper] = (subvariant1[i],"Opt")
                posInSuper+=1
        
    # Variant 2 has remaining activities while variant 1 does not
    elif(currentIndex2<len(subvariant2)-1 and currentIndex1==len(subvariant1)-1):
        for i in range(startIndex2,len(subvariant2)):
            result[posInSuper] = (subvariant2[i],"Opt")
            posInSuper+=1
            
    else:
        # In this case, neither one or the other ending subprocesses is optional
        # we must summarize the underlying process
        return result

    return result
        

In [61]:
optionalVariant = OptionalActivity(["F","A","B","C","E"],["A","B","C","E","G","H"])
print(optionalVariant)

The variants both contain the following common activities: {'A': (1, 0), 'B': (2, 1), 'C': (3, 2), 'E': (4, 3)}
Non-matching preceeding subprocess of variant 1: ['F']
Non-matching preceeding subprocess of variant 2: []
Non-matching preceeding subprocess of variant 1: []
Non-matching preceeding subprocess of variant 2: []
Non-matching preceeding subprocess of variant 1: []
Non-matching preceeding subprocess of variant 2: []
Non-matching preceeding subprocess of variant 1: []
Non-matching preceeding subprocess of variant 2: []
{0: ('F', 'Opt'), 1: ('A', 'NOpt'), 2: ('B', 'NOpt'), 3: ('C', 'NOpt'), 4: ('E', 'NOpt'), 5: ('G', 'Opt'), 6: ('H', 'Opt')}
